# Weighted Forecasting for Time Series Store forecast 
* By Alex Dance https://www.linkedin.com/in/alex-dance/
* This notebook is one of several notebooks for a project to improve store and product forecasts
1.	EDA – Exploratory Data Analysis – includes working with annual forecasts
2.	Main Modelling
3.	XG Boost modelling by Month
4.	Weighted average
5.	ARIMA – Month and Other Modelling
6.	Deep Learning

* This forecast does not use a model but rather weightings and average weightings multiplied by average sales to give a forecast

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error , mean_absolute_error

In [3]:
class color:  # Testing to make the heading look a liitle more impressive
   BOLD = '\033[1m'

In [4]:
df = pd.read_csv(r'C:/Users/alexd/Alex Folder 1/Capstone - Store Forecast/train.csv')
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [5]:
originaldf = pd.read_csv(r'C:/Users/alexd/Alex Folder 1/Capstone - Store Forecast/train.csv')

In [6]:
split = "2017-01-01"

In [7]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [8]:
df['date'] =  pd.to_datetime(df['date'])

# Function for Error calculations

In [9]:
def calculate_errorb(test_sales,  test_prediction):
    MSE_test = mean_squared_error(y_true=test_sales,  y_pred=test_prediction) # Mean Square Error (MAE)
    MAE_test = mean_absolute_error(y_true=test_sales,  y_pred=test_prediction) # Mean Absolute Error (MAE)
    MAPE = np.mean(np.abs(test_prediction - test_sales  ) **2)  # Mean Absolute Percentage Error (MAPE)
    RMSE  = np.mean(np.sqrt((test_prediction - test_sales) ** 2))    
    return{'MSE_test': MSE_test ,'MAE_test': MAE_test,  'MAPE':MAPE, 'RMSE':RMSE}

# Feature Engineering

In [10]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [11]:
split = "2016-12-31"

In [12]:
df['ItemStoreCombined'] = df['item'].map(str) + '-' + df['store'].map(str) 

In [13]:
df.head()

,date,store,item,sales,ItemStoreCombined
0,2013-01-01,1,1,13,1-1
1,2013-01-02,1,1,11,1-1
2,2013-01-03,1,1,14,1-1
3,2013-01-04,1,1,13,1-1
4,2013-01-05,1,1,10,1-1


In [14]:
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofyear'] = df['date'].dt.dayofyear
df['dayofmonth'] = df['date'].dt.day
df['weekofyear'] = df['date'].dt.weekofyear

In [15]:
df = df.set_index('date')

In [16]:
df.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,,,
2013-01-01,1,1,13,1-1,1,1,1,2013,1,1,1
2013-01-02,1,1,11,1-1,2,1,1,2013,2,2,1
2013-01-03,1,1,14,1-1,3,1,1,2013,3,3,1
2013-01-04,1,1,13,1-1,4,1,1,2013,4,4,1
2013-01-05,1,1,10,1-1,5,1,1,2013,5,5,1


In [17]:
df_train = df[ :split ] 

In [18]:
df_test = df[split : ] 

In [19]:
df_train.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,,,
2013-01-01,1,1,13,1-1,1,1,1,2013,1,1,1
2013-01-02,1,1,11,1-1,2,1,1,2013,2,2,1
2013-01-03,1,1,14,1-1,3,1,1,2013,3,3,1
2013-01-04,1,1,13,1-1,4,1,1,2013,4,4,1
2013-01-05,1,1,10,1-1,5,1,1,2013,5,5,1


In [20]:
df_test_final = df_test.copy()

In [21]:
df_test_final =df_test_final.drop (['dayofweek', 'quarter','month', 'year', 'dayofyear', 'weekofyear'],axis=1)

In [22]:
train_cols=list(df_train.columns)

In [23]:
print(train_cols)

['store', 'item', 'sales', 'ItemStoreCombined', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear']


In [24]:
df_train = df_train.loc[:,train_cols] #XXXXXXX

# Same for Test

In [25]:
test_cols=list(df_test.columns)

In [26]:
df_test = df_test.loc[:,test_cols] #XXXXXXX

# Now Working on Training review

In [27]:
df_train['Calculated_year'] =  df_train.index.year - min(df_train.index.year) + 1

In [28]:
df_train.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year
date,,,,,,,,,,,,
2013-01-01,1,1,13,1-1,1,1,1,2013,1,1,1,1
2013-01-02,1,1,11,1-1,2,1,1,2013,2,2,1,1
2013-01-03,1,1,14,1-1,3,1,1,2013,3,3,1,1
2013-01-04,1,1,13,1-1,4,1,1,2013,4,4,1,1
2013-01-05,1,1,10,1-1,5,1,1,2013,5,5,1,1


In [29]:
df_train['Calculated_year'].value_counts()

4    183000
3    182500
2    182500
1    182500
Name: Calculated_year, dtype: int64

# Now working through the weightings
* This is a solution that a lot of standard forecasts would be completed, often in excel
* This solution is completed to see how it compares
* In summary it shows how weightings of the month and then aply that weightings across a total forecast
* Multiple different weightings can be applied
* With such a high number of rows this would be impractical using Excel

In [30]:
month_weighting= (( df_train.groupby(['month']).agg([np.nanmean]).sales - np.nanmean(df_train.sales) ) / np.nanmean(df_train.sales)).rename(columns={'nanmean':'month_weighting'})
df_train=df_train.join(month_weighting,how='left',on='month')

In [31]:
df_train.tail()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year,month_weighting
date,,,,,,,,,,,,,
2016-12-27,10,50,60,50-10,1,4,12,2016,362,27,52,4,-0.249279
2016-12-28,10,50,43,50-10,2,4,12,2016,363,28,52,4,-0.249279
2016-12-29,10,50,68,50-10,3,4,12,2016,364,29,52,4,-0.249279
2016-12-30,10,50,63,50-10,4,4,12,2016,365,30,52,4,-0.249279
2016-12-31,10,50,64,50-10,5,4,12,2016,366,31,52,4,-0.249279


In [32]:
month_weighting.head(13)

,month_weighting
month,
1,-0.319326
2,-0.246433
3,-0.093797
4,0.053017
5,0.135099
6,0.206043
7,0.281457
8,0.132709
9,0.052223


In [33]:
year_weighting= (( df_train.groupby(['year']).agg([np.nanmean]).sales - np.nanmean(df_train.sales) ) /  np.nanmean(df_train.sales)).rename(columns={'nanmean':'year_weighting'})

In [34]:
print(year_weighting)

      year_weighting
year                
2013       -0.140220
2014       -0.010923
2015        0.032537
2016        0.118282


In [35]:
CAGR = 0.096 #only for using on the train data - can be adjusted

In [36]:
year_weighting.loc[6,:] =  np.mean(CAGR)*3
df_train=df_train.join(year_weighting,how='left',on='year')

In [37]:
weekday_weighting= ( ( df_train.groupby(['dayofweek']).agg([np.nanmean]).sales - np.nanmean(df_train.sales) ) /  np.nanmean(df_train.sales)).rename(columns={'nanmean':'weekday_weighting'})
df_train=df_train.join(weekday_weighting,how='left',on='dayofweek')

In [38]:
store_item_weighting= ( ( df_train.groupby(['store','item']).agg([np.nanmean]).sales - np.nanmean(df_train.sales) ) / np.nanmean(df_train.sales)).rename(columns={'nanmean':'store_item_weighting'})
df_train=df_train.join(store_item_weighting,how='left',on=['store','item'])

In [39]:
df_train['product_combined_weighting']=np.product(df_train.loc[:,['month_weighting','year_weighting','weekday_weighting','store_item_weighting',]]+1,axis=1)

In [40]:
df_train.sample()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year,month_weighting,year_weighting,weekday_weighting,store_item_weighting,product_combined_weighting
date,,,,,,,,,,,,,,,,,
2014-03-22,1,21,32,21-1,5,1,3,2014,81,22,12,2,-0.093797,-0.010923,0.122128,-0.307967,0.696025


In [41]:
df_train.tail()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year,month_weighting,year_weighting,weekday_weighting,store_item_weighting,product_combined_weighting
date,,,,,,,,,,,,,,,,,
2016-12-27,10,50,60,50-10,1,4,12,2016,362,27,52,4,-0.249279,0.118282,-0.076576,0.419121,1.100146
2016-12-28,10,50,43,50-10,2,4,12,2016,363,28,52,4,-0.249279,0.118282,-0.074857,0.419121,1.102194
2016-12-29,10,50,68,50-10,3,4,12,2016,364,29,52,4,-0.249279,0.118282,-0.011256,0.419121,1.177967
2016-12-30,10,50,63,50-10,4,4,12,2016,365,30,52,4,-0.249279,0.118282,0.056470,0.419121,1.258654
2016-12-31,10,50,64,50-10,5,4,12,2016,366,31,52,4,-0.249279,0.118282,0.122128,0.419121,1.336878


In [42]:
df_train.Calculated_year.nunique()

4

In [43]:
df_train.Calculated_year.value_counts()

4    183000
3    182500
2    182500
1    182500
Name: Calculated_year, dtype: int64

In [44]:
print(weekday_weighting)

           weekday_weighting
dayofweek                   
0                  -0.207027
1                  -0.076576
2                  -0.074857
3                  -0.011256
4                   0.056470
5                   0.122128
6                   0.191042


In [45]:
print(month_weighting)

       month_weighting
month                 
1            -0.319326
2            -0.246433
3            -0.093797
4             0.053017
5             0.135099
6             0.206043
7             0.281457
8             0.132709
9             0.052223
10           -0.019591
11            0.057928
12           -0.249279


In [46]:
print(store_item_weighting)

            store_item_weighting
store item                      
1     1                -0.616305
      2                 0.017385
      3                -0.364471
      4                -0.616427
      5                -0.682033
...                          ...
10    46                0.266392
      47               -0.527397
      48                0.106590
      49               -0.365553
      50                0.419121

[500 rows x 1 columns]


In [47]:
df_train.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year,month_weighting,year_weighting,weekday_weighting,store_item_weighting,product_combined_weighting
date,,,,,,,,,,,,,,,,,
2013-01-01,1,1,13,1-1,1,1,1,2013,1,1,1,1,-0.319326,-0.14022,-0.076576,-0.616305,0.207355
2013-01-02,1,1,11,1-1,2,1,1,2013,2,2,1,1,-0.319326,-0.14022,-0.074857,-0.616305,0.207741
2013-01-03,1,1,14,1-1,3,1,1,2013,3,3,1,1,-0.319326,-0.14022,-0.011256,-0.616305,0.222022
2013-01-04,1,1,13,1-1,4,1,1,2013,4,4,1,1,-0.319326,-0.14022,0.056470,-0.616305,0.237230
2013-01-05,1,1,10,1-1,5,1,1,2013,5,5,1,1,-0.319326,-0.14022,0.122128,-0.616305,0.251974


In [48]:
df_train['sales_prediction']=np.round(df_train.product_combined_weighting*np.round(np.nanmean(df_train.sales),1))  

In [49]:
average_train_sales = np.nanmean(df_train.sales)
print(average_train_sales)

50.61022861054072


In [50]:
df_train.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Calculated_year,month_weighting,year_weighting,weekday_weighting,store_item_weighting,product_combined_weighting,sales_prediction
date,,,,,,,,,,,,,,,,,,
2013-01-01,1,1,13,1-1,1,1,1,2013,1,1,1,1,-0.319326,-0.14022,-0.076576,-0.616305,0.207355,10.0
2013-01-02,1,1,11,1-1,2,1,1,2013,2,2,1,1,-0.319326,-0.14022,-0.074857,-0.616305,0.207741,11.0
2013-01-03,1,1,14,1-1,3,1,1,2013,3,3,1,1,-0.319326,-0.14022,-0.011256,-0.616305,0.222022,11.0
2013-01-04,1,1,13,1-1,4,1,1,2013,4,4,1,1,-0.319326,-0.14022,0.056470,-0.616305,0.237230,12.0
2013-01-05,1,1,10,1-1,5,1,1,2013,5,5,1,1,-0.319326,-0.14022,0.122128,-0.616305,0.251974,13.0


# now apply the weightings to the test sales

In [51]:
df_test=df_test.join(month_weighting,how='left',on='month')

In [52]:
df_test['Calculated_year'] =  5

In [53]:
year_weighting_17 =0.22  # calculated seperately

In [54]:
df_test['year_weighting'] = year_weighting_17

In [55]:
df_test=df_test.join(weekday_weighting,how='left',on='dayofweek')

In [56]:
df_test=df_test.join(store_item_weighting,how='left',on=['store','item'])

In [57]:
df_test.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,month_weighting,Calculated_year,year_weighting,weekday_weighting,store_item_weighting
date,,,,,,,,,,,,,,,,
2016-12-31,1,1,14,1-1,5,4,12,2016,366,31,52,-0.249279,5,0.22,0.122128,-0.616305
2017-01-01,1,1,19,1-1,6,1,1,2017,1,1,52,-0.319326,5,0.22,0.191042,-0.616305
2017-01-02,1,1,15,1-1,0,1,1,2017,2,2,1,-0.319326,5,0.22,-0.207027,-0.616305
2017-01-03,1,1,10,1-1,1,1,1,2017,3,3,1,-0.319326,5,0.22,-0.076576,-0.616305
2017-01-04,1,1,16,1-1,2,1,1,2017,4,4,1,-0.319326,5,0.22,-0.074857,-0.616305


In [58]:
df_test['smry_product']=np.product(df_test.loc[:,['month_weighting','year_weighting','weekday_weighting','store_item_weighting',]]+1,axis=1)

In [59]:
df_test['weighted_sales_prediction']=df_test.smry_product*average_train_sales

In [60]:
average_train_sales

50.61022861054072

In [61]:
df_test.sum()

store                                                                  1006500
item                                                                   4666500
sales                                                                 10757532
ItemStoreCombined            1-11-11-11-11-11-11-11-11-11-11-11-11-11-11-11...
dayofweek                                                               551500
quarter                                                                 460000
month                                                                  1197000
year                                                                 369110500
dayofyear                                                             33580500
dayofmonth                                                             2884500
weekofyear                                                             4875000
month_weighting                                                       -93.8354
Calculated_year                                     

In [62]:
df_test.head()

,store,item,sales,ItemStoreCombined,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,month_weighting,Calculated_year,year_weighting,weekday_weighting,store_item_weighting,smry_product,weighted_sales_prediction
date,,,,,,,,,,,,,,,,,,
2016-12-31,1,1,14,1-1,5,4,12,2016,366,31,52,-0.249279,5,0.22,0.122128,-0.616305,0.394336,19.957456
2017-01-01,1,1,19,1-1,6,1,1,2017,1,1,52,-0.319326,5,0.22,0.191042,-0.616305,0.379501,19.206616
2017-01-02,1,1,15,1-1,0,1,1,2017,2,2,1,-0.319326,5,0.22,-0.207027,-0.616305,0.252664,12.787394
2017-01-03,1,1,10,1-1,1,1,1,2017,3,3,1,-0.319326,5,0.22,-0.076576,-0.616305,0.294230,14.891030
2017-01-04,1,1,16,1-1,2,1,1,2017,4,4,1,-0.319326,5,0.22,-0.074857,-0.616305,0.294777,14.918743


In [63]:
RMSE_weighted  = np.mean(np.sqrt((df_test['weighted_sales_prediction'] - df_test['sales']) ** 2)) 
print(RMSE_weighted)

6.462572198330651
